In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/home/scott/Documents/git/neukrill-net-work


In [3]:
import sys
import numpy as np
import skimage
import cv2
import sklearn

In [4]:
import imp

In [5]:
from holoviews import *

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [6]:
import neukrill_net.utils
import neukrill_net.highlevelfeatures

In [7]:
import time

In [8]:
settings = neukrill_net.utils.Settings('settings.json')

In [9]:
X,y = settings.flattened_train_paths(settings.classes)

In [10]:
import mahotas

In [11]:
H = mahotas.features.haralick(neukrill_net.highlevelfeatures.loadimage(X[0]))

In [12]:
H

array([[  5.96246215e-01,   9.36057994e+01,   8.69212478e-01,
          3.57854472e+02,   8.40299018e-01,   5.04100052e+02,
          1.33781209e+03,   1.74656209e+00,   2.13745597e+00,
          2.44471960e-03,   1.50417321e+00,  -2.97456891e-01,
          7.25368324e-01],
       [  5.90407904e-01,   1.65499469e+02,   7.72179871e-01,
          3.63223984e+02,   8.36613763e-01,   5.04009284e+02,
          1.28739647e+03,   1.79006532e+00,   2.19116187e+00,
          2.41530385e-03,   1.55429730e+00,  -2.75972162e-01,
          7.10040310e-01],
       [  5.96714914e-01,   8.52161669e+01,   8.80720868e-01,
          3.57213225e+02,   8.39870373e-01,   5.04108475e+02,
          1.34363673e+03,   1.75212804e+00,   2.13801150e+00,
          2.44732258e-03,   1.51517479e+00,  -2.98367275e-01,
          7.26307520e-01],
       [  5.88269882e-01,   1.25390716e+02,   8.27392020e-01,
          3.63223984e+02,   8.32801944e-01,   5.04009284e+02,
          1.32750522e+03,   1.81663231e+00,   2.229

In [13]:
H.shape

(4, 13)

In [14]:
(np.amax(H, 0) - np.amin(H, 0)).shape

(13,)

In [15]:
imp.reload(neukrill_net.highlevelfeatures)

<module 'neukrill_net.highlevelfeatures' from '/home/scott/Documents/git/neukrill-net-tools/neukrill_net/highlevelfeatures.pyc'>

In [16]:
hlf = neukrill_net.highlevelfeatures.Haralick()

In [17]:
hlf.extract_image(neukrill_net.highlevelfeatures.loadimage(X[0]))

array([  5.92909729e-01,   1.17428038e+02,   8.37376309e-01,
         3.60378916e+02,   8.37396275e-01,   5.04056774e+02,
         1.32408763e+03,   1.77634694e+00,   2.17408130e+00,
         2.42847737e-03,   1.54530917e+00,  -2.79362396e-01,
         7.10802264e-01,   8.44503234e-03,   8.02833026e+01,
         1.08540997e-01,   6.01075856e+00,   7.49707356e-03,
         9.91907566e-02,   5.62402684e+01,   7.00702220e-02,
         9.22398670e-02,   4.07591384e-05,   1.03418161e-01,
         5.27140192e-02,   4.48146178e-02])

In [18]:
XF = hlf.transform(X)

In [18]:
XF.shape

(1, 30336, 26)

In [19]:
XF = numpy.squeeze(XF,0)

In [20]:
XF.shape

(30336, 26)

### Stochastic Gradient Descent Classifier

In [29]:
clf = sklearn.linear_model.SGDClassifier(loss='log', penalty='elasticnet')

In [30]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(XF.squeeze(0), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))

Time=1.36498403549
Accuracy=0.0768064345992


/home/scott/Documents/git/neukrill-venv-auto/local/lib/python2.7/site-packages/sklearn/linear_model/base.py:231: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/scott/Documents/git/neukrill-venv-auto/local/lib/python2.7/site-packages/sklearn/linear_model/base.py:238: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
settings.class_priors.shape

(121,)

In [32]:
clf.predict_proba(X_test).shape

(15168, 121)

In [37]:
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, 0.5*settings.class_priors + 0.5*clf.predict_proba(X_test))))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_test.shape

In [ ]:
sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))

In [ ]:
np.count_nonzero(np.isnan(X_test))

In [ ]:
np.count_nonzero(np.isinf(X_test))

In [ ]:
np.isfinite(X_test).all()

In [ ]:
X_test.max()

In [ ]:
np.isfinite(X_test.sum())

In [ ]:
clf.predict_proba(X_test)

In [ ]:
X_test[-2:,:]

In [ ]:
p = clf.predict_proba(X_test)

In [ ]:
X_test.shape, p.shape

In [ ]:
p[1:10,:]

In [ ]:
np.count_nonzero(np.isnan(p))

In [ ]:
t0 = time.time()
sklearn.cross_validation.cross_val_score(clf, XF, y, scoring=sklearn.metrics.log_loss, cv=9)
t1 = time.time()
total = t1-t0
print("Time={}".format(total))

### Random forest

In [19]:
import sklearn.ensemble

In [41]:
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5)

In [42]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(XF.squeeze(0), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=135.16323781
Accuracy=0.439345991561
Logloss=2.20661963322


In [45]:
t0 = time.time()
cvscores = sklearn.cross_validation.cross_val_score(clf, XF.squeeze(0), y=y, score_func=sklearn.metrics.log_loss, cv=9)
print(cvscores)
t1 = time.time()
total = t1-t0
print("Time={}".format(total))

/home/scott/Documents/git/neukrill-venv-auto/local/lib/python2.7/site-packages/sklearn/cross_validation.py:1143: DeprecationWarning: Passing function as ``score_func`` is deprecated and will be removed in 0.15. Either use strings or score objects. The relevant new parameter is called ''scoring''.
  scorer = check_scoring(estimator, score_func=score_func, scoring=scoring)


ValueError: y_true and y_pred have different number of classes 121, 2

### Logistic Regression

In [51]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

In [39]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(XF.squeeze(0), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=77.6372170448
Accuracy=0.331355485232
Logloss=2.7772124244


Forgot to scale, and that is important for Logistic Regression!

In [52]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=32.7626669407
Accuracy=0.418380801688
Logloss=2.27816887117


Now we're about as good as the Random Forest!

### Naive Bayes

In [47]:
import sklearn.naive_bayes

In [48]:
clf = sklearn.naive_bayes.GaussianNB()

In [49]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(XF.squeeze(0), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=0.0631310939789
Accuracy=0.276239451477
Logloss=8.5176239126


Well that sucked. Maybe better with a Z-score?

In [50]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=0.0487160682678
Accuracy=0.275975738397
Logloss=8.57971640574


Nope. Naive Bayes just plain sucks.

### SVM

In [53]:
clf = sklearn.svm.LinearSVC()

In [54]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=47.1796929836
Accuracy=0.426885548523


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

No predict probabilities opiton with SVC 

### Results so far
We have very similar results for `LogisticRegression` and the `RandomForest`. However, `LogisticRegression` could still be optimised with a grid search.

Let's do proper CV

In [55]:
cv = sklearn.cross_validation.StratifiedShuffleSplit(y)

In [56]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

In [ ]:
print('Cross-validating')
results = []
for train, test in cv:
    sc = sklearn.preprocessing.StandardScaler()
    sc.fit(XF.squeeze(0)[train,:])
    clf.fit(sc.transform(XF.squeeze(0)[train,:]), np.array(y)[train])
    res = sklearn.metrics.log_loss(np.array(y)[test], clf.predict_proba(sc.transform(XF.squeeze(0)[test,:])))
    print(res)
    results.append(res)

Cross-validating
19.906186849

In [ ]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(XF.squeeze(0), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

In [ ]:
sklearn.grid_search.GridSearchCV(estimator, param_grid, scoring=None)